In [ ]:
import pandas as pd
import numpy as np
import datetime
import plotly.graph_objects as go
import warnings; warnings.simplefilter('ignore')
from IPython.display import display, Markdown, Image, SVG
import re
import bmondata
from bmondata import Server
from dateutil.relativedelta import relativedelta
import scrapbook as sb

In [ ]:
# Parameters to be changed/exported using Papermill or Scrapbook
org_id = 0
server_web_address = 'https://bms.ahfc.us'

In [33]:
# 'Glue' down variables for scrapbook to export later
sb.glue('sort_order', 6)
sb.glue('title', 'Historical Org. Energy Comp')

server = Server(server_web_address)

# Write test to make sure there is more than one building for comparison.

def calc_monthly_elec_cost(electric_rates, monthly_kwh, fifteen_min_peak):
        if electric_rates['block1'] is None:
            monthly_usage_cost = electric_rates['customer_charge'] + electric_rates['rate1'] * monthly_kwh
            monthly_demand_charges = electric_rates['demand_charge'] * fifteen_min_peak
            total_monthly_cost = monthly_usage_cost + monthly_demand_charges
        elif isinstance(electric_rates['block1'], float):
                if monthly_kwh > electric_rates['block1']: 
                    block_1_cost = electric_rates['block1'] * electric_rates['rate1']
                    remaining_usage = monthly_kwh - electric_rates['block1']
                    remainder_cost = remaining_usage * electric_rates['rate2']
                    monthly_usage_cost = electric_rates['customer_charge'] + block_1_cost + remainder_cost
                    monthly_demand_charges = electric_rates['demand_charge'] * fifteen_min_peak
                    total_monthly_cost = monthly_usge_cost + monthly_demand_charges
                elif monthly_kwh <= electric_rates['block1']:
                    monthly_usage_cost = electric_rates['customer_charge'] + electric_rates['rate1'] * monthly_kwh
                    monthly_demand_charges = electric_rates['demand_charge'] * fifteen_min_peak
                    total_monthly_cost = monthly_usage_cost + monthly_demand_charges
        else: 
            print ('Cost calculation error')

        return (total_monthly_cost, monthly_usage_cost, monthly_demand_charges)

# Need to do this for buildings that don't have cost data -- see the organizational_annual_comparisons_V4 for ideas
def calculate_eui(server, building_id, start_date, end_date):
    
    building_name = server.buildings(building_id)[0]['title']
    building_df = server.buildings(building_id)
    electric_id = building_df[0]['electric_ids']
    
    # Get hourly average kW
    hourly_usage_building = server.sensor_readings((electric_id, 'monthly_electric_usage_kwh'),
                           start_ts =  start_date,
                           end_ts = end_date,
                           averaging='1H')
    
    # Calculate monthly total kWh usage
    monthly_usage_building = hourly_usage_building.groupby([lambda x: x.year, lambda x: x.month]).sum()
    monthly_usage_building = monthly_usage_building.reset_index()
    monthly_usage_building = monthly_usage_building.rename(columns={'level_0': 'year','level_1':'month'})
    
    ### Calculate the hourly fuel usage rate
    fuel_ids = building_df[0]['fuel_ids']
    hourly_fuel_usage = server.sensor_readings((fuel_ids, 'fuel_usage'),
                               start_ts = start_date, end_ts = end_date,
                               averaging='1H')
    
    # Calculate the monthly total fuel usage and cost
    monthly_fuel = hourly_fuel_usage.groupby([lambda x: x.year, lambda x: x.month]).sum()
    monthly_fuel['monthly_fuel_ccf'] = monthly_fuel.fuel_usage / 100000.0
    monthly_fuel = monthly_fuel.reset_index()
    monthly_fuel = monthly_fuel.rename(columns={'level_0': 'year','level_1':'month'})
    
    # Combine the fuel and electricity usage data
    monthly_total_energy = pd.merge(monthly_fuel, monthly_usage_building, how='outer',
                                      left_on=['year', 'month'], right_on=['year', 'month'])
    
    # Calculate the annual ECI, Electric EUI, and Fuel EUI
    annual_intensity_numbers = monthly_total_energy.groupby(['year']).sum()
    annual_intensity_numbers['building_sqft'] = server.buildings(building_id)[0]['floor_area']
    
    annual_intensity_numbers['electric_eui'] = annual_intensity_numbers.monthly_electric_usage_kwh / annual_intensity_numbers.building_sqft
    annual_intensity_numbers['fuel_eui'] = annual_intensity_numbers.fuel_usage / 1000.0 / annual_intensity_numbers.building_sqft
    annual_intensity_numbers['building_name'] = building_name
    
    annual_intensity_numbers = annual_intensity_numbers.reset_index()
    
    return annual_intensity_numbers

def calculate_eui_eci(server, building_id, start_date, end_date):
    
    building_name = server.buildings(building_id)[0]['title']
    electric_rates = server.buildings(building_id)[0]['electric_rate']
    building_df = server.buildings(building_id)
    electric_id = building_df[0]['electric_ids']
    
    # Get hourly average kW
    hourly_usage_building = server.sensor_readings((electric_id, 'monthly_electric_usage_kwh'),
                           start_ts =  start_date,
                           end_ts = end_date,
                           averaging='1H')
    
    # Calculate monthly total kWh usage
    monthly_usage_building = hourly_usage_building.groupby([lambda x: x.year, lambda x: x.month]).sum()
    monthly_usage_building = monthly_usage_building.reset_index()
    monthly_usage_building = monthly_usage_building.rename(columns={'level_0': 'year','level_1':'month'})
    
    # Get the fifteen minute average to calculate demand chrages
    fifteen_minute_usage_building = server.sensor_readings((electric_id, 'fifteen_minute_usage_kw'),
                                                      start_ts = start_date, end_ts = end_date,
                                                      averaging = '15min')

    # Get the highest fifteen minute average kW used per month to calculate the demand charges
    grouped_fifteen_minute_demand = fifteen_minute_usage_building.groupby([lambda x: x.year, lambda x: x.month]).max()
    grouped_fifteen_minute_demand = grouped_fifteen_minute_demand.reset_index()
    grouped_fifteen_minute_demand = grouped_fifteen_minute_demand.rename(columns={'level_0': 'year',
                                                                                 'level_1':'month'})
    
    # Combine the electric demand data with the monthly electric usage data
    monthly_electric_data = pd.merge(grouped_fifteen_minute_demand, monthly_usage_building, how='outer',
                                 left_on=['year', 'month'], right_on=['year', 'month'])
    
    # Create blank columns to fill in the for loop below
    monthly_electric_data['total_electric_cost'] = np.nan
    monthly_electric_data['monthly_kwh_cost'] = np.nan
    monthly_electric_data['monthly_demand_charges'] = np.nan
    
    # Calculate the monthly electrical cost  including customer, demand, and per kWh charges
    for i in np.arange(0, monthly_electric_data.index.shape[0]):
        (monthly_electric_data.at[i,'total_electric_cost'], 
        monthly_electric_data.at[i,'monthly_kwh_cost'],
        monthly_electric_data.at[i,'monthly_demand_charges']) = calc_monthly_elec_cost(electric_rates, 
                                                                                      monthly_electric_data.at[i, 'monthly_electric_usage_kwh'],
                                                                                      monthly_electric_data.at[i, 'fifteen_minute_usage_kw'])

    ### Calculate the hourly fuel usage rate
    fuel_rates = server.buildings(building_id)[0]['fuel_rate']
    fuel_ids = building_df[0]['fuel_ids']
    hourly_fuel_usage = server.sensor_readings((fuel_ids, 'fuel_usage'),
                               start_ts = start_date, end_ts = end_date,
                               averaging='1H')
    
    # Calculate the monthly total fuel usage and cost
    monthly_fuel = hourly_fuel_usage.groupby([lambda x: x.year, lambda x: x.month]).sum()
    monthly_fuel['monthly_fuel_ccf'] = monthly_fuel.fuel_usage / 100000
    monthly_fuel['monthly_fuel_cost'] = monthly_fuel.monthly_fuel_ccf * fuel_rates['rate'] + fuel_rates['customer_charge']
    monthly_fuel = monthly_fuel.reset_index()
    monthly_fuel = monthly_fuel.rename(columns={'level_0': 'year','level_1':'month'})
    
    # Combine the fuel and electricity usage and cost data
    monthly_total_energy_costs = pd.merge(monthly_fuel, monthly_electric_data, how='outer',
                                      left_on=['year', 'month'], right_on=['year', 'month'])
    monthly_total_energy_costs['total_monthly_costs'] = monthly_total_energy_costs.total_electric_cost + monthly_total_energy_costs.monthly_fuel_cost
    
    # Calculate the annual ECI, Electric EUI, and Fuel EUI
    annual_intensity_numbers = monthly_total_energy_costs.groupby(['year']).sum()
    annual_intensity_numbers['building_sqft'] = server.buildings(building_id)[0]['floor_area']
    
    # Use full years only
    annual_intensity_numbers = annual_intensity_numbers.query("month == 78")
    
    annual_intensity_numbers['eci'] = annual_intensity_numbers.total_monthly_costs / annual_intensity_numbers.building_sqft
    annual_intensity_numbers['electric_eui'] = annual_intensity_numbers.monthly_electric_usage_kwh / annual_intensity_numbers.building_sqft
    annual_intensity_numbers['fuel_eui'] = annual_intensity_numbers.fuel_usage / 1000 / annual_intensity_numbers.building_sqft
    annual_intensity_numbers['building_name'] = building_name
    annual_intensity_numbers = annual_intensity_numbers.reset_index()
    
    return annual_intensity_numbers

all_buildings = server.buildings()
org_df = pd.DataFrame(all_buildings)
# change organizatioon data into pandas columns and then merge back in to original dataframe
organizations = org_df.organizations.apply(lambda x: pd.Series(x))
org_columns = organizations[0].apply(pd.Series)
org_columns = org_columns.rename(columns={0: 'organization_id',
                                         1: 'organization_name'})
org_df = pd.merge(org_df, org_columns, how='left',
                 left_index=True, right_index=True)

if org_id == 0:
    organization_name = 'all organizations on the server found at ' + server_web_address
else:
    organization_name = org_df.query("organization_id == @org_id").organization_name.iloc[0]

def check_data_for_buildings_in_org(org_df, org_id):
    ''' Checks a dataframe with the building metadata for an entire organization to
    determine if there is sufficient data for each building to calculate an EUI/ECI. 
    Returns a list of building ids for all buildings in the organization with sufficient
    data.'''
    if org_id == 0:
        org_df = org_df
    else:
        org_df = org_df.query("organization_id == @org_id")
        
    buildings_with_sufficient_data = pd.DataFrame(columns=['building_id', 'cost_data'])
    counter = 0
    for building_id in org_df.id.unique():
        
        building_df = org_df.query("id == @building_id")
        
        insufficient_data = 0 
        
        if building_df.floor_area.isna().iloc[0]:
            insufficient_data = 1 
        elif building_df['fuel_ids'].empty or building_df['fuel_ids'].iloc[0] == '':
            insufficient_data = 1 
        elif building_df['electric_ids'].empty or building_df['electric_ids'].iloc[0] == '':
            insufficient_data = 1 
        
        if insufficient_data == 0:
            cost_data = 1

            if building_df.fuel_rate.iloc[0] is None:
                cost_data = 0
            elif building_df.electric_rate.iloc[0] is None:
                cost_data = 0
            
            buildings_with_sufficient_data.at[counter, 'building_id'] = building_id
            buildings_with_sufficient_data.at[counter, 'cost_data'] = cost_data
            counter += 1
            
    return buildings_with_sufficient_data

buildings_with_data = check_data_for_buildings_in_org(org_df, org_id)

if buildings_with_data.empty:
    error_message = 'There are less than two buildings with more than one year of data to show trending.'
    raise RuntimeError(error_message)

for idx, row in buildings_with_data.iterrows():
    building_id = row['building_id']
    building_df = org_df.query("id == @building_id")
    electric_dates = server.sensor_readings(building_df['electric_ids'].values[0]).index
    timedelta = relativedelta(electric_dates.max(), electric_dates.min())
    months_of_data = timedelta.years * 12 + timedelta.months
    buildings_with_data.at[idx, 'months_of_data'] = months_of_data

buildings_with_data['at_least_two_years'] = np.where(buildings_with_data.months_of_data >= 24, 1, 0)

if buildings_with_data.query("at_least_two_years == 1").shape[0] < 2:
    error_message = 'There are less than two buildings with more than one year of data to show trending.'
    raise RuntimeError(error_message)

if buildings_with_data.cost_data.sum() != buildings_with_data.shape[0]:
    all_building_euis = []
    for building_id_i in buildings_with_data.building_id.unique():
        building_df = calculate_eui(server, 
                                    building_id_i, 
                                    datetime.datetime.now() - relativedelta(years=10), 
                                    datetime.datetime.now())
        all_building_euis.append(building_df)
        
    all_building_df = pd.concat(all_building_euis, axis=0)
    
else:
    all_building_eui_and_ecis = []
    for building_id_i in buildings_with_data.building_id.unique():
        building_df = calculate_eui_eci(server, 
                                        building_id_i, 
                                        datetime.datetime.now() - relativedelta(years=10), 
                                        datetime.datetime.now())
        all_building_eui_and_ecis.append(building_df)
        
    all_building_df = pd.concat(all_building_eui_and_ecis, axis=0)

all_building_df = all_building_df.rename(columns={'monthly_fuel_cost':'annual_fuel_cost',
                                                 'monthly_electric_usage_kwh':'annual_electric_usage_kwh',
                                                 'monthly_fuel_ccf':'annual_fuel_use_ccf',
                                                  'total_monthly_costs':'total_annual_energy_costs'
                                                 })

all_building_df['annual_fuel_usage_mmbtu'] = all_building_df.fuel_usage.div(1000000.0)

all_building_df = all_building_df.query("month == 78")

diverging_hues = ['#d73027','#fc8d59',
                  '#fee090','#ffffbf',
                  '#e0f3f8','#91bfdb',
                  '#4575b4']

if 'eci' in all_building_df.columns:
    md_title2 = '''## Historical Trend: Energy Cost and Consumption Per Square Foot for {:s}'''
    md_title2 = md_title2.format(organization_name)
else: 
    md_title2 = '''## Historical Trend: Energy Consumption Per Square Foot for {:s}'''
    md_title2 = md_title2.format(organization_name)

#################################################################################################

In [ ]:
Markdown(md_title2)

In [ ]:
if 'eci' in all_building_df.columns:
    counter = 0
    building_i_list = []

    for building_i in all_building_df.building_name.unique():

        building_i_df = all_building_df.query("building_name == @building_i")

        building_eci = go.Scatter(x=building_i_df.year,
                             y=building_i_df.eci,
                             name=building_i,
                             line=dict(color=diverging_hues[counter]))

        building_i_list.append(building_eci)
        counter += 1

    layout = dict(title= 'ECI comparison for ' + organization_name + ' buildings',
                  xaxis=dict(title='Year', tickformat='d'),
                  yaxis=dict(title='Energy Cost Index ($/square foot/year)')
                 )


    fig = go.Figure(dict(data=building_i_list, layout=layout))
    fig.show()

In [ ]:
counter = 0

building_i_list = []

for building_i in all_building_df.building_name.unique():
    
    building_i_df = all_building_df.query("building_name == @building_i")
    
    building_electric_eui = go.Scatter(x=building_i_df.year,
                         y=building_i_df.electric_eui,
                         name=building_i,
                         line=dict(color=diverging_hues[counter]))
    
    building_i_list.append(building_electric_eui)
    counter += 1

layout = dict(title= 'Electric EUI comparison for ' + organization_name + ' buildings',
              xaxis=dict(title='Year', tickformat='d'),
              yaxis=dict(title='Electricity energy use intensity (kWh / square foot / year)')
             )


fig = go.Figure(dict(data=building_i_list, layout=layout))
fig.show()

In [ ]:
counter = 0

building_i_list = []

for building_i in all_building_df.building_name.unique():
    
    building_i_df = all_building_df.query("building_name == @building_i")
    
    building_fuel_eui = go.Scatter(x=building_i_df.year,
                         y=building_i_df.fuel_eui,
                         name=building_i,
                         line=dict(color=diverging_hues[counter]))
    
    building_i_list.append(building_fuel_eui)
    counter += 1

layout = dict(title= 'Fuel EUI comparison for ' + organization_name + ' buildings',
              xaxis=dict(title='Year', tickformat='d'),
              yaxis=dict(title='Fuel energy use intensity (kBtu / square foot / year)')
             )


fig = go.Figure(dict(data=building_i_list, layout=layout))
fig.show()

In [ ]:
if 'eci' in all_building_df.columns:
    md_title2 = '''## Historical Trend: Energy Cost and Consumption Totals for {:s}'''
    md_title2 = md_title2.format(organization_name)
else: 
    md_title2 = '''## Historical Trend: Energy Consumption Totals for {:s}'''
    md_title2 = md_title2.format(organization_name) 

In [ ]:
Markdown(md_title2)

In [ ]:
counter = 0

building_i_list = []

for building_i in all_building_df.building_name.unique():
    
    building_i_df = all_building_df.query("building_name == @building_i")
    
    building_fuel_eui = go.Scatter(x=building_i_df.year,
                         y=building_i_df.annual_electric_usage_kwh,
                         name=building_i,
                         line=dict(color=diverging_hues[counter]))
    
    building_i_list.append(building_fuel_eui)
    counter += 1

layout = dict(title= 'Annual Electric Consumption Comparison for ' + organization_name + ' Buildings',
              xaxis=dict(title='Year', tickformat='d'),
              yaxis=dict(title='Electricity Consumption (kWh / year)')
             )


fig = go.Figure(dict(data=building_i_list, layout=layout))
fig.show()

In [ ]:
counter = 0

building_i_list = []

for building_i in all_building_df.building_name.unique():
    
    building_i_df = all_building_df.query("building_name == @building_i")
    
    building_fuel_eui = go.Scatter(x=building_i_df.year,
                         y=building_i_df.annual_fuel_usage_mmbtu,
                         name=building_i,
                         line=dict(color=diverging_hues[counter]))
    
    building_i_list.append(building_fuel_eui)
    counter += 1

layout = dict(title= 'Annual Fuel Consumption Comparison for ' + organization_name + ' Buildings',
              xaxis=dict(title='Year', tickformat='d'),
              yaxis=dict(title='Fuel Consumption (Millions of BTUs / year)')
             )


fig = go.Figure(dict(data=building_i_list, layout=layout))
fig.show()

In [ ]:
if 'annual_energy_costs' in all_building_df.columns:
    counter = 0

    building_i_list = []

    for building_i in all_building_df.building_name.unique():

        building_i_df = all_building_df.query("building_name == @building_i")

        building_fuel_eui = go.Scatter(x=building_i_df.year,
                             y=building_i_df.total_annual_energy_costs,
                             name=building_i,
                             line=dict(color=diverging_hues[counter]))

        building_i_list.append(building_fuel_eui)
        counter += 1

    layout = dict(title= 'Total Annual Energy Cost Comparison for ' + organization_name + ' Buildings',
                  xaxis=dict(title='Year', tickformat='d'),
                  yaxis=dict(title='Total Annual Energy Costs ($ / year)')
                 )


    fig = go.Figure(dict(data=building_i_list, layout=layout))
    fig.show()